##### 导入库

In [414]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn import tree
from collections import Counter
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SVMSMOTE,SMOTE
from joblib import dump,load
from sklearn.model_selection import GridSearchCV

##### 选择数据

In [415]:
n='N_all.csv'
p='P_all.csv'
p='P_ice.csv'
p='P_her.csv'
p='P_mix.csv'
# p='P_ice1.csv'
way="mix"


##### 读取数据

In [416]:
fea_dict={
    'all':[
  "minvalue",
  "retang",
  "fft_abs_22",
  "fft_real_2",
  "lumpiness",
  "fft_real_5",
  "fft_angle_3",
  "alt_stderr_min_50",
  "fft_imag_6",
  "erbc_num10_focus2",
  "fft_imag_3",
  "alt_rvalue_min_50"
],


 'ice':[
  "retang",
  "erbc_num10_focus4",
  "fft_abs_22",
  "fft_real_2",
  "minindex",
  "alt_rvalue_min_50",
  "F2",
  "fft_imag_6",
  "asoc",
],

 'her':[
"fft_real_2",
"fft_real_5",
"dmq",
"fft_angle_3",
"erbc_num10_focus2",
"fft_imag_3",
"alt_rvalue_min_50"
],

'mix':[
"minvalue",
"cv",
"lumpiness",
"alt_rvalue_min_50",
"fft_angle_3",
"alt_stderr_min_50",
"maxindex"
]
}

fea_list=fea_dict[way]

def read_data(tit):
    path = 'E:\\desktop\\data\\app\\fea\\'
    #读取数据
    data = pd.read_csv(path+tit,encoding='gbk')
    data = data.set_index('name')
    return data

n_fea=read_data(n)
p_fea=read_data(p)

n_fea["tag"]=[0 for i in range(len(n_fea))]
p_fea["tag"]=[1 for i in range(len(p_fea))]

print(len(n_fea),len(p_fea))


523 75


##### 过采样

In [417]:
#过采样
over=SMOTE(random_state=0)
# over=SVMSMOTE(random_state=0)
np_fea=shuffle(pd.concat([n_fea,p_fea],axis=0))
x_data_=np_fea[fea_list]
y_data_=np_fea["tag"]

x_data,y_data=over.fit_resample(x_data_,y_data_)

#切分
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3)
print(x_data.shape)

(1046, 7)


In [418]:
def jugde(y_pred,y_true):
    tn,fn,tp,fp=0,0,0,0
    for i in range(len(y_pred)):
        
        if y_pred[i]==y_true[i]:
            if y_true[i]:
                tp+=1
            else:
                tn+=1
        else:
            if y_true[i]:
                fp+=1
            else:
                fn+=1

    print("检出率： ",tp,"/",tp+fp,' ',round(tp*100/(tp+fp),2),"%")
    print("误判率： ",fn,"/",tn+fn,' ',round(fn*100/(tn+fn),2),"%")
    print("精确率： ",(tn+tp),"/",(tn+fp+tp+fp),' ',round((tn+tp)*100/(tn+fp+tp+fp),2),"%")


##### 搭建模型

In [419]:
etc=ExtraTreesClassifier(random_state=0)
etc.fit(x_train,y_train)
y_pred=etc.predict(x_test)

print("测试集中：")
jugde(y_pred,list(y_test))

print("所有数据中：")
y_pred=etc.predict(x_data_)
jugde(y_pred,list(y_data_))

etc.score(x_test,y_test)

      minvalue        cv  lumpiness  alt_rvalue_min_50  fft_angle_3  \
677   0.467616  0.635118   0.425456           0.421369     0.487048   
343   0.269224  0.998369   0.680464           0.418423     0.257296   
966   0.567514  0.524213   0.310846           0.404649     0.511186   
845   0.550461  0.249609   0.120246           0.588397     0.463326   
425   0.703754  0.281212   0.045694           0.527916     0.630328   
...        ...       ...        ...                ...          ...   
224   0.510855  0.500465   0.168171           0.452289     0.576785   
475   0.211702  0.635247   0.067314           0.851781     0.728998   
52    0.449426  0.487368   0.680464           0.175407     0.000000   
128   0.588714  0.590317   0.478882           0.376205     0.334731   
1014  0.541462  0.411575   0.120589           0.473891     0.748917   

      alt_stderr_min_50  maxindex  
677            0.705624  0.511562  
343            1.000000  0.400000  
966            0.641835  0.400000  
845

0.8853503184713376

In [420]:
# #保存模型
# dump(etc,'etc_'+way+".joblib")
# print("保存成功")